In [1]:
#IMPORTS
import os
import pandas as pd
import chardet
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling
import sys

from pathlib import Path

directory = Path(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.append(str(directory))

mammal_path = os.path.join(directory, 'ecoimpactmapper/ecocomponents/mammals')
invertebrate_path = os.path.join(directory, 'ecoimpactmapper/ecocomponents/invertebrates')
fish_path = os.path.join(directory, 'ecoimpactmapper/ecocomponents/fish') # Not yet implemented
bird_path = os.path.join(directory, 'ecoimpactmapper/ecocomponents/birds') # Not yet implemented

aoi_path = os.path.join(directory, 'data', 'vectors', 'international', 'aoi.geojson')
aoi = gpd.read_file(aoi_path)
aoi = aoi.to_crs(epsg=4326)





In [2]:
def create_tif_from_gdf(gdf, output_tif, value_column='Overall Probability', cell_size_x=0.5, cell_size_y=0.5):
    # Get bounds from the GeoDataFrame
    minx, miny, maxx, maxy = gdf.total_bounds
    
    # Set resolution based on the desired cell size
    xres = cell_size_x
    yres = cell_size_y

    # Create an empty raster array
    num_cols = int((maxx - minx) / xres) + 1
    num_rows = int((maxy - miny) / yres) + 1
    raster = np.zeros((num_rows, num_cols), dtype=np.float32)

    # Map the points to the raster grid
    for _, row in gdf.iterrows():
        col = int((row.geometry.x - minx) / xres)
        row_idx = int((maxy - row.geometry.y) / yres)
        raster[row_idx, col] = row[value_column]

    # Define the affine transform
    transform = from_origin(minx, maxy, xres, yres)

    # Create and save the raster file
    with rasterio.open(
        output_tif,
        'w',
        driver='GTiff',
        height=raster.shape[0],
        width=raster.shape[1],
        count=1,
        dtype=raster.dtype,
        crs=gdf.crs,
        transform=transform,
    ) as dst:
        dst.write(raster, 1)


def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    return chardet.detect(raw_data)['encoding']


def load_csvs_into_dict(root_folder):
    dataframes_dict = {}
    
    for subdir, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.csv') and file != 'template_df.csv':
                file_path = os.path.join(subdir, file)
                try:
                    df = pd.read_csv(file_path, skiprows=6, header=1, encoding='ISO-8859-1')
                    dataframes_dict[file] = df
                except Exception as e:
                    print(f"Error reading {file}: {str(e)}")
    
    return dataframes_dict


def convert_to_geodataframe(dataframes_dict, aoi, clip = True):
    gdfs_dict = {}
    
    for file_name, df in dataframes_dict.items():
        try:
            geometry = [Point(xy) for xy in zip(df['Center Long'], df['Center Lat'])]
            gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

            # Clip to the AOI
            gdf = gpd.clip(gdf, aoi) if clip else gdf
            
            
            gdfs_dict[file_name.split('.')[0]] = gdf

        except Exception as e:
            print(f"Error converting {file_name} to GeoDataFrame: {str(e)}")
    
    return gdfs_dict


def overlay_points_on_raster(input_tif, gdf, output_tif):
    # Read the existing raster
    with rasterio.open(input_tif) as src:
        raster_data = src.read(1)
        transform = src.transform
        crs = src.crs
        width = src.width
        height = src.height
        bounds = src.bounds

    # Create an empty raster array for the output
    new_raster_data = np.zeros_like(raster_data, dtype=np.uint8)

    # Map the points to the raster grid
    for _, row in gdf.iterrows():
        # Get the x and y coordinates of the point
        x, y = row.geometry.x, row.geometry.y

        # Convert the coordinates to column and row indices in the raster
        col, row_idx = ~transform * (x, y)
        col, row_idx = int(col), int(row_idx)

        # Ensure the indices are within the raster bounds
        if 0 <= col < width and 0 <= row_idx < height:
            new_raster_data[row_idx, col] = 1

    # Save the new raster
    with rasterio.open(
        output_tif,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=new_raster_data.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(new_raster_data, 1)


def sum_rasters_in_folder(folder_path, output_tif):
    # Get a list of all raster files in the folder
    raster_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.tif')]

    if not raster_files:
        raise ValueError("No raster files found in the specified folder.")

    # Read the first raster to get metadata
    with rasterio.open(raster_files[0]) as src:
        profile = src.profile
        data_sum = np.zeros((src.height, src.width), dtype=np.float32)

    # Sum all rasters
    for raster_file in raster_files:
        with rasterio.open(raster_file) as src:
            data = src.read(1)
            data_sum += data

    # Update the profile to reflect the number of layers in the output raster
    profile.update(dtype=rasterio.float32, count=1)

    # Save the sum raster
    with rasterio.open(output_tif, 'w', **profile) as dst:
        dst.write(data_sum, 1)


In [5]:
# Invertebrates

raster_folder = os.path.join(invertebrate_path, 'rasters')

if not os.path.exists(raster_folder):
    os.makedirs(raster_folder)

# loop through the files in the folder
for file in os.listdir(invertebrate_path):

    if not file.endswith('.csv'):
        continue

    print(file)
    file_path = os.path.join(invertebrate_path, file)
    template_df = pd.read_csv(file_path, header=3, encoding='ISO-8859-1')
    geometry = [Point(xy) for xy in zip(template_df['Center Longitude'], template_df['Center Latitude'])]
    template_gdf = gpd.GeoDataFrame(template_df, crs="EPSG:4326", geometry=geometry)

    template_gdf = gpd.clip(template_gdf, aoi)

    output_tif = os.path.join(raster_folder, f"{os.path.splitext(file)[0]}.tif")
    create_tif_from_gdf(template_gdf, output_tif, value_column='Species Count', cell_size_x=0.5, cell_size_y=0.5)


sum_rasters_in_folder(raster_folder, os.path.join(raster_folder, 'invertebrates_sum.tif'))

anthropoda.csv
cnidaria.csv
echinodermata.csv
mollusca.csv


In [6]:
# Mammals

raster_folder = os.path.join(mammal_path, 'rasters')
template_raster = os.path.join(raster_folder, 'template_raster.tif')



if not os.path.exists(raster_folder):
    os.makedirs(raster_folder)

# Create the template raster
template_csv = os.path.join(mammal_path, 'template_df.csv')

template_df = pd.read_csv(template_csv, header=8, encoding='ISO-8859-1')
geometry = [Point(xy) for xy in zip(template_df['Center Long'], template_df['Center Lat'])]
template_gdf = gpd.GeoDataFrame(template_df, crs="EPSG:4326", geometry=geometry)

template_gdf = gpd.clip(template_gdf, aoi)

create_tif_from_gdf(template_gdf, template_raster, value_column='Overall Probability')


# loop through the files in the folder
df_dict = load_csvs_into_dict(mammal_path)
gdf_dict = convert_to_geodataframe(df_dict, aoi)

for file_name, gdf in gdf_dict.items():
    output_tif = f'/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/mammals/rasters/{file_name}.tif'
    overlay_points_on_raster(template_raster, gdf, output_tif)

sum_rasters_in_folder(raster_folder, os.path.join(mammal_path, 'mammals_sum.tif'))

In [7]:
## FISH FROM REDLIST

fish_path = '/Users/loucas/Downloads/redlist_species_data_1ed23a96-60e2-40dd-b5f3-003ef20493f0/data_0.shp'

fish_gdf = gpd.read_file(fish_path)

# clip to aoi
fish_gdf = gpd.clip(fish_gdf, aoi)

# remove the largest polygon as it is just the aoi for some reason
fish_gdf = fish_gdf[fish_gdf.geometry.area != fish_gdf.geometry.area.max()]


/opt/anaconda3/envs/NOCN/lib/python3.9/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
/var/folders/mk/ks8rs6mx6_30xy34h6hr7cs00000gn/T/ipykernel_62121/3741913678.py:11: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  fish_gdf = fish_gdf[fish_gdf.geometry.area != fish_gdf.geometry.area.max()]


In [8]:
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np

# Assuming your GeoDataFrame is called 'gdf'
# and you have a path to your template raster

# Step 1: Open the template raster
with rasterio.open('/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/mammals/mammals_sum.tif') as src:
    template_meta = src.meta.copy()
    template_transform = src.transform
    template_crs = src.crs
    template_shape = src.shape

# Step 2: Create an empty raster with the same properties as the template
raster = np.zeros(template_shape, dtype=np.int32)

# Step 3: Prepare shapes for rasterization
shapes = ((geom, 1) for geom in fish_gdf.geometry)

# Step 4: Rasterize the polygons, counting each polygon once per cell
rasterized = features.rasterize(
    shapes=shapes,
    out=raster,
    transform=template_transform,
    all_touched=False,  # Changed to False
    merge_alg=rasterio.enums.MergeAlg.add
)

# Step 5: Clip the values to the maximum number of polygons
max_polygons = len(fish_gdf)
rasterized = np.clip(rasterized, 0, max_polygons)

# Step 6: Save the raster to a file
output_meta = template_meta.copy()
output_meta.update({
    'dtype': 'int32',
    'count': 1
})

with rasterio.open('/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/fish.tif', 'w', **output_meta) as dst:
    dst.write(raster, 1)

In [14]:
# Convert all tifs to 3035 and normalize between 0 and 1

import os
from osgeo import gdal, osr
import numpy as np

def normalize_and_reproject(input_file, output_file):
    # Open the input file
    ds = gdal.Open(input_file)
    
    # Read the data as a NumPy array
    data = ds.ReadAsArray().astype(np.float32)
    
    # Replace NaN values with 0
    data = np.nan_to_num(data, nan=0.0)
    
    # Normalize the data between 0 and 1
    min_val = np.min(data)
    max_val = np.max(data)
    if min_val != max_val:
        normalized_data = (data - min_val) / (max_val - min_val)
    else:
        normalized_data = data  # Avoid division by zero if all values are the same
    
    # Get the geotransform and projection of the input file
    geotransform = ds.GetGeoTransform()
    projection = ds.GetProjection()
    
    # Check if the input file has a defined CRS
    if not projection:
        # If no CRS is defined, set it to EPSG:4326 (WGS84)
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)
        projection = srs.ExportToWkt()
        print(f"No CRS found for {input_file}. Setting to EPSG:4326.")
    
    # Create the output dataset
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_file, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(geotransform)
    out_ds.SetProjection(projection)
    
    # Write the normalized data to the output file
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(normalized_data)
    
    # Close the datasets
    ds = None
    out_ds = None
    
    # Reproject to EPSG:3035
    output_3035 = output_file.replace('.tif', '_3035.tif')
    gdal.Warp(output_3035, output_file, dstSRS='EPSG:3035')
    
    # Remove the intermediate file
    os.remove(output_file)
    
    print(f"Processed {input_file} -> {output_3035}")

def process_folder(input_folder):
    for filename in os.listdir(input_folder):
        if filename.endswith('.tif'):
            input_file = os.path.join(input_folder, filename)
            output_file = os.path.join(input_folder, f'normalized_{filename}')
            normalize_and_reproject(input_file, output_file)


input_folder = '/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test'
process_folder(input_folder)

Processed /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test/phytoplankton.tif -> /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test/normalized_phytoplankton_3035.tif
Processed /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test/fish.tif -> /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test/normalized_fish_3035.tif
Processed /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test/invertebrates_sum.tif -> /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test/normalized_invertebrates_sum_3035.tif
Processed /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test/mammals_sum.tif -> /Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test/normalized_mammals_sum_3035.tif
Processed /Users/loucas/Documents/ORG/github/marine-planning/ecoim

In [ ]:
# THIS STUFF IS FOR GETTING THE COMPONENT DATA INTO THE CSV FOR THE ECOIMPACT MAPPER FORMAT


import os
import rasterio
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter

def generate_ecoimpact_map(shape, num_clusters=10, cluster_size=20, smoothness=8):
    # Your existing function code here
    map_array = np.zeros(shape)
    
    for _ in range(num_clusters):
        center = tuple(np.random.randint(0, dim) for dim in shape)
        value = np.random.uniform(2.0, 5.0)
        
        for i in range(-cluster_size, cluster_size + 1):
            for j in range(-cluster_size, cluster_size + 1):
                x = (center[0] + i) % shape[0]
                y = (center[1] + j) % shape[1]
                map_array[x, y] = value
    
    smoothed_map = gaussian_filter(map_array, sigma=smoothness)
    smoothed_map = (smoothed_map - smoothed_map.min()) / (smoothed_map.max() - smoothed_map.min()) * 3 + 2
    return np.round(smoothed_map, decimals=2)


def folder_tifs_to_dataframe(folder_path):
    tif_files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]
    if not tif_files:
        raise ValueError("No TIF files found in the specified folder.")

    dataframes = []
    
    for tif_file in tif_files:
        tif_path = os.path.join(folder_path, tif_file)
        base_name = os.path.splitext(tif_file)[0]
        column_name = base_name
        stressor_column_name = f"{base_name}_stressor"
        
        with rasterio.open(tif_path) as src:
            raster_data = src.read(1)
            transform = src.transform
            
            # Generate stressor map
            stressor_data = generate_ecoimpact_map(raster_data.shape)

        x_coords = []
        y_coords = []
        values = []
        stressor_values = []

        rows, cols = raster_data.shape
        for row in range(rows):
            for col in range(cols):
                x, y = rasterio.transform.xy(transform, row + 0.5, col + 0.5)
                value = raster_data[row, col]
                stressor_value = stressor_data[row, col]
                
                x_coords.append(x)
                y_coords.append(y)
                values.append(value)
                stressor_values.append(stressor_value)

        df = pd.DataFrame({
            'X': x_coords,
            'Y': y_coords,
            column_name: values,
            #stressor_column_name: stressor_values
        })
        
        dataframes.append(df)

    # Merge all dataframes
    result_df = dataframes[0]
    for df in dataframes[1:]:
        result_df = pd.merge(result_df, df, on=['X', 'Y'])

    return result_df

tif_path = '/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test'


df = folder_tifs_to_dataframe(tif_path)

df.to_csv('/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/test.csv', index=False)

In [41]:
## CONVERT THE RESULTING ECO SENSITIVITY MAPS TO TIF

import os
import numpy as np
import rasterio
from rasterio.transform import from_origin

def convert_txt_to_tif(input_file, output_file, cell_size=5000):
    # Read data from txt file
    with open(input_file, 'r') as f:
        next(f)  # Skip the header line
        data = [line.strip().split(',') for line in f]  # Assuming comma-separated values
    
    x = np.array([float(row[0]) for row in data])
    y = np.array([float(row[1]) for row in data])
    values = np.array([float(row[2]) for row in data])

    # Determine raster properties
    # Adjust x_min and y_min to represent the top-left corner of the raster
    x_min, x_max = x.min() - cell_size, x.max()
    y_min, y_max = y.min(), y.max() + cell_size
    
    # Calculate width and height, adding 1 to include the rightmost and topmost cells
    width = int(np.ceil((x_max - x_min) / cell_size))
    height = int(np.ceil((y_max - y_min) / cell_size))

    # Create an empty raster
    raster = np.full((height, width), np.nan, dtype=np.float32)

    # Fill the raster with values
    # Adjust the indices calculation to account for bottom-right corner coordinates
    col_indices = np.clip(((x - x_min) / cell_size - 1).astype(int), 0, width - 1)
    row_indices = np.clip((height - 1 - (y - y_min) / cell_size).astype(int), 0, height - 1)
    raster[row_indices, col_indices] = values

    # Create the GeoTIFF
    # Use x_min and y_max for the origin as they now represent the top-left corner
    transform = from_origin(x_min, y_max, cell_size, cell_size)
    with rasterio.open(
        output_file,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=1,
        dtype=raster.dtype,
        crs='EPSG:3035',
        transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(raster, 1)


# Directory containing the txt files
input_dir = '/Users/loucas/Documents/ORG/github/marine-planning/ecoimpactmapper/ecocomponents/monopile'

# Process all txt files in the directory
for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        input_file = os.path.join(input_dir, filename)
        output_file = os.path.join(input_dir, filename.replace('.txt', '.tif'))
        convert_txt_to_tif(input_file, output_file)
        print(f'Converted {filename} to {filename.replace(".txt", ".tif")}')


Converted eco_sens_wind.txt to eco_sens_wind.tif
Converted eco_sens_aquaculture.txt to eco_sens_aquaculture.tif
Converted eco_sens_fpv.txt to eco_sens_fpv.tif
